In [10]:
import pickle
import numpy as np

for beta in np.linspace(0, 1, 21, endpoint=True):
    fname = 'sqIBv1-%0.5f' % beta
    try:
        with open('outdata/%s'%fname, 'rb') as f:
            saved_data = pickle.load(f)
            print('beta: %0.4f Ixt: %0.4f Iyt: %0.4f' % (beta, saved_data[-1]['trn_Ixt'], saved_data[-1]['trn_Iyt']))
    except:
        print(fname, 'not found')

beta: 0.0000 Ixt: 10.9506 Iyt: 2.2532
beta: 0.0500 Ixt: 2.6300 Iyt: 2.2166
beta: 0.1000 Ixt: 2.4193 Iyt: 2.0532
beta: 0.1500 Ixt: 2.0273 Iyt: 1.7528
beta: 0.2000 Ixt: 1.2976 Iyt: 1.1523
beta: 0.2500 Ixt: 0.0001 Iyt: 0.0015
beta: 0.3000 Ixt: 1.2083 Iyt: 1.0022
beta: 0.3500 Ixt: 0.7163 Iyt: 0.6382
beta: 0.4000 Ixt: 0.6381 Iyt: 0.5793
beta: 0.4500 Ixt: 0.6823 Iyt: 0.6374
beta: 0.5000 Ixt: 0.4609 Iyt: 0.3767
beta: 0.5500 Ixt: 0.6101 Iyt: 0.5112
beta: 0.6000 Ixt: 0.0001 Iyt: 0.0016
beta: 0.6500 Ixt: 0.3305 Iyt: 0.3093
beta: 0.7000 Ixt: 0.3381 Iyt: 0.2929
beta: 0.7500 Ixt: 0.0001 Iyt: 0.0014
beta: 0.8000 Ixt: 0.3275 Iyt: 0.3064
beta: 0.8500 Ixt: 0.0001 Iyt: 0.0016
beta: 0.9000 Ixt: 0.3254 Iyt: 0.3022
beta: 0.9500 Ixt: 0.0001 Iyt: -0.0014
beta: 1.0000 Ixt: 0.3216 Iyt: 0.3092


In [2]:
saved_data

[{'epoch': 0,
  'beta': 0.0,
  'log_sigma2': -4.0,
  'log_eta2': 2.4249346,
  'trn_acc': 0.9115,
  'tst_acc': 0.905,
  'trn_loss': -1.946672,
  'tst_loss': -1.9431643,
  'trn_Ixt': 9.436941,
  'tst_Ixt': 9.432081,
  'trn_Iyt': 1.946672,
  'tst_Iyt': 1.9431643},
 {'epoch': 1,
  'beta': 0.0,
  'log_sigma2': -4.0,
  'log_eta2': 1.7379284,
  'trn_acc': 0.93583333,
  'tst_acc': 0.934,
  'trn_loss': -2.0430593,
  'tst_loss': -2.0121531,
  'trn_Ixt': 9.278589,
  'tst_Ixt': 9.246666,
  'trn_Iyt': 2.0430593,
  'tst_Iyt': 2.0121531},
 {'epoch': 2,
  'beta': 0.0,
  'log_sigma2': -4.0,
  'log_eta2': 0.99433696,
  'trn_acc': 0.9458333,
  'tst_acc': 0.936,
  'trn_loss': -2.084187,
  'tst_loss': -2.0595202,
  'trn_Ixt': 8.798514,
  'tst_Ixt': 8.742575,
  'trn_Iyt': 2.084187,
  'tst_Iyt': 2.0595202},
 {'epoch': 3,
  'beta': 0.0,
  'log_sigma2': -4.0,
  'log_eta2': 2.5835326,
  'trn_acc': 0.9605,
  'tst_acc': 0.951,
  'trn_loss': -2.1429777,
  'tst_loss': -2.122617,
  'trn_Ixt': 9.457356,
  'tst_Ixt': 